In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

#fname = "root://eospublic.cern.ch//eos/opendata/cms/Run2016H/DoubleMuon/NANOAOD/UL2016_MiniAODv2_NanoAODv9-v1/2510000/127C2975-1B1C-A046-AABF-62B77E757A86.root"
fname = "file:/opendata_cms/DoubleMuon/Run2016H-UL2016_MiniAODv2_NanoAODv9-v1/NANOAOD/127C2975-1B1C-A046-AABF-62B77E757A86.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema,
    metadata={"dataset": "DoubleMuon"},
).events()

print("Number of events = ", ak.num(events.Muon, axis=0).compute())

/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/usr/local/venv/lib/python3.10/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cr

Number of events =  2147195


In [2]:
print("Number of muons = ", ak.count(events.Muon.mass, axis=None).compute())

Number of muons =  4806013


In [3]:
selected_muons = events.Muon[(events.Muon.isGlobal == 1)]
print("Number of selected muons after isGlobal==1: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after isGlobal==1:  4110800


In [4]:
selected_muons = selected_muons[(selected_muons.nStations > 0)]
print("Number of selected muons after nStations>0: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nStations>0:  3936437


In [5]:
selected_muons = selected_muons[(selected_muons.nTrackerLayers > 5)]
print("Number of selected muons after nTrackerLayers>5: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after nTrackerLayers>5:  3908889


In [6]:
selected_muons = selected_muons[(selected_muons.dxy < 0.3) & (selected_muons.dz < 20.0)]
print("Number of selected muons after PV impact param sel: ", ak.count(selected_muons.mass, axis=None).compute())       

Number of selected muons after PV impact param sel:  3892547


In [7]:
selected_muons = selected_muons[(selected_muons.pt > 3.0) & (abs(selected_muons.eta) < 2.4)]
print("Number of selected muons after pt,eta sel: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after pt,eta sel:  3883500


In [8]:
selected_muons = selected_muons[(selected_muons.pfRelIso03_chg < 0.5)]
print("Number of selected muons after ptcone30 isolation: ", ak.count(selected_muons.mass, axis=None).compute())

Number of selected muons after ptcone30 isolation:  2681124


In [9]:
selectede_muons = selected_muons[ak.num(selected_muons) > 0]
print("Number of selected events with num(muons)>0: ", ak.num(selectede_muons.mass, axis=0).compute())
print("Corresponding number of selected muons: ", ak.count(selectede_muons.mass, axis=None).compute())

Number of selected events with num(muons)>0:  1604214
Corresponding number of selected muons:  2681124


In [10]:
#selected_muons_c = selected_muons.compute()
pass

In [11]:
fourmuon = ak.combinations(selected_muons, 4, fields=['lep1','lep2','lep3','lep4'])

In [12]:
print(type(fourmuon))

<class 'dask_awkward.lib.core.Array'>


In [13]:
print("Number of 4-muon combination: ", ak.count(fourmuon.lep1.mass, axis=None).compute())

Number of 4-muon combination:  6170


In [14]:
fourmuon = fourmuon[(fourmuon.lep1.charge + fourmuon.lep2.charge == 0) &
                    (fourmuon.lep3.charge + fourmuon.lep4.charge == 0)]

In [15]:
print("Number of 4-muon combination after charge check: ", ak.count(fourmuon.lep1.mass, axis=None).compute())

Number of 4-muon combination after charge check:  2304


In [16]:
fourmuon["pair1"] = fourmuon.lep1 + fourmuon.lep2
fourmuon["pair2"] = fourmuon.lep3 + fourmuon.lep4